In [2]:
import pandas as pd
import numpy as np
import os

In [6]:
dfs_raw = {}
for m in ['chi','phi','nyc','sfba']:
    print(f'{m}..')
    dfs_raw[m] = pd.read_csv(f'../data/modeling_files/{m}_regression_file.csv')
    dfs_raw[m]['fips12_d'] = dfs_raw[m]['fips12_d'].astype('str').str.pad(12, 'left',fillchar='0')
    dfs_raw[m]['fips12_o'] = dfs_raw[m]['fips12_o'].astype('str').str.pad(12, 'left',fillchar='0')


chi..
phi..
nyc..
sfba..


In [7]:
import pickle
bg_dists = {}
for m in ['chi','phi','sfba','nyc']:
    print(f'{m}..')
    with open(f'../data/temp/bg_dists_{m}', 'rb') as fp:
        bg_dists[m] = pickle.load(fp)

chi..
phi..
sfba..
nyc..


In [8]:
#for m in ['chi','phi','sfba','nyc']:
dfs_add = {}
for m in ['chi','nyc','phi','sfba']:
    print(f'{m}..')
    dfs_add[m] = dfs_raw[m].merge(bg_dists[m].rename(columns={'value':'dist_raw'})[['dist_raw']].reset_index(), left_on=['fips12_o','fips12_d'], right_on=['fips12_o','fips12_d'], how='left')
    dfs_add[m]['dist_raw_km'] = dfs_add[m]['dist_raw']/1000


chi..
nyc..
phi..
sfba..


In [9]:
dfs_add['chi']['veh_pp_o']

0          0.428073
1          0.428073
2          0.428073
3          0.380645
4          0.380645
             ...   
6086201    0.560872
6086202    0.560872
6086203    0.560872
6086204    0.560872
6086205    0.560872
Name: veh_pp_o, Length: 6086206, dtype: float64

In [10]:
import statsmodels.api as sm

#FEATS = ['extra_zonal','dist_km', 'dist_raw_km', 'medinc_Z_o', 'medinc_Z_d', 'BG_active_share', 'BG_transit_share']
#FEATS = ['extra_zonal', 'dist_raw_km', 'medinc_Z_o', 'medinc_Z_d', 'BG_active_share', 'BG_transit_share']
FEATS = ['extra_zonal', 'dist_raw_km', 'medinc_Z_o', 'medinc_Z_d', 'veh_commute_pp_o','veh_pp_o']
TARGET = 'active_transit'

#models = {}
models_no_bg_share = {}
for m in ['chi','phi','nyc','sfba']:
    print(f'{m}..')
    df_modeling = dfs_add[m][FEATS+[TARGET]].dropna()

    X = df_modeling[FEATS]
    y = df_modeling[TARGET]

    logit_model = sm.Logit(y,X)
    result = logit_model.fit()

    models_no_bg_share[m] = {
    #models[m] = {
        'model': logit_model,
        'result': result,
        'summary': result.summary()
    }

chi..
Optimization terminated successfully.
         Current function value: 0.528766
         Iterations 6
phi..
Optimization terminated successfully.
         Current function value: 0.624023
         Iterations 5
nyc..
Optimization terminated successfully.
         Current function value: 0.680901
         Iterations 4
sfba..
Optimization terminated successfully.
         Current function value: 0.522828
         Iterations 6


In [20]:
FEATS = ['extra_zonal', 'dist_raw_km', 'medinc_Z_o', 'medinc_Z_d', 'veh_commute_pp_o','veh_pp_o', 'metro_o']
df_modeling = pd.concat(dfs_add)[FEATS+[TARGET]].dropna()
X = df_modeling[FEATS]
X = pd.get_dummies(X, dtype=float)
y = df_modeling[TARGET]

logit_model = sm.Logit(y,X)
result = logit_model.fit()

Optimization terminated successfully.
         Current function value: 0.573016
         Iterations 6


In [21]:
total_summary = result.summary()

<class 'statsmodels.iolib.summary.Summary'>
"""
                           Logit Regression Results                           
==============================================================================
Dep. Variable:         active_transit   No. Observations:             22605599
Model:                          Logit   Df Residuals:                 22605589
Method:                           MLE   Df Model:                            9
Date:                Mon, 03 Feb 2025   Pseudo R-squ.:                  0.1419
Time:                        22:09:29   Log-Likelihood:            -1.2953e+07
converged:                       True   LL-Null:                   -1.5096e+07
Covariance Type:            nonrobust   LLR p-value:                     0.000
====================================================================================
                       coef    std err          z      P>|z|      [0.025      0.975]
------------------------------------------------------------------------------------
extra_zonal          0.0803      0.002     47.740      0.000       0.077       0.084
dist_raw_km         -0.3609      0.000   -749.365      0.000      -0.362      -0.360
medinc_Z_o           0.1507      0.001    275.492      0.000       0.150       0.152
medinc_Z_d           0.0219      0.001     41.661      0.000       0.021       0.023
veh_commute_pp_o    -5.7224      0.008   -695.226      0.000      -5.738      -5.706
veh_pp_o            -1.4070      0.005   -305.218      0.000      -1.416      -1.398
metro_o_chi          1.9234      0.002    816.132      0.000       1.919       1.928
metro_o_nyc          1.6869      0.001   1306.766      0.000       1.684       1.689
metro_o_phi          1.9847      0.002    824.636      0.000       1.980       1.989
metro_o_sfba         1.8798      0.002    765.454      0.000       1.875       1.885
====================================================================================
"""

In [ ]:
# for m in ['chi','nyc','phi','sfba']:
#     models[m]['summary'] = models[m]['result'].summary()
dfs_raw['sfba'][dfs_raw['sfba']['fips12_o'] == dfs_raw['sfba']['fips12_d']]

,activity_id,fips12_o,fips12_d,primary_mode,zoneID_o,zoneID_d,active_transit,dist_km,extra_zonal,metro_o,...,medinc_Z_o,metro_d,fips5_d,total_pop_d,medinc_d,veh_pp_d,veh_commute_pp_d,medinc_Z_d,BG_active_share,BG_transit_share
0,7.620967e+18,060014001002,060014001002,private_auto,10,10,0.0,2.747799,0,sfba,...,3.007879,sfba,6001,1167,242813,0.768584,0.205236,2.865596,0.246915,0.024839
1,1.819653e+18,060014001002,060014001002,private_auto,10,10,0.0,0.672111,0,sfba,...,3.007879,sfba,6001,1167,242813,0.768584,0.205236,2.865596,0.246915,0.024839
2,6.288796e+18,060014002001,060014002001,private_auto,10,10,0.0,2.235547,0,sfba,...,3.007879,sfba,6001,904,226094,0.674779,0.138557,2.534651,0.246915,0.024839
3,4.348066e+18,060014002001,060014002001,private_auto,10,10,0.0,1.661275,0,sfba,...,3.007879,sfba,6001,904,226094,0.674779,0.138557,2.534651,0.246915,0.024839
4,2.660973e+18,060014216002,060014216002,private_auto,10,10,0.0,2.901338,0,sfba,...,3.007879,sfba,6001,1413,228919,0.656759,0.198922,2.590571,0.246915,0.024839
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
7373473,6.514097e+18,060855130005,060855130005,auto_passenger,31,31,0.0,1.339158,0,sfba,...,-1.090776,sfba,6085,2922,51793,0.178688,0.087850,-0.915555,0.338369,0.012657
7373474,1.667297e+19,060855130005,060855130005,biking,31,31,1.0,1.954880,0,sfba,...,-1.090776,sfba,6085,2922,51793,0.178688,0.087850,-0.915555,0.338369,0.012657
7373475,1.445997e+19,060855130005,060855130005,private_auto,31,31,0.0,1.907926,0,sfba,...,-1.090776,sfba,6085,2922,51793,0.178688,0.087850,-0.915555,0.338369,0.012657
7373476,1.200712e+19,060855123081,060855123081,walking,0,0,1.0,1.543591,0,sfba,...,0.601597,sfba,6085,3590,250001,0.829248,0.301703,3.007879,0.063342,0.000000


In [70]:
# models['sfba']['summary']
# models['chi']['summary']
# models['phi']['summary']
# models['nyc']['summary']

<class 'statsmodels.iolib.summary.Summary'>
"""
                           Logit Regression Results                           
==============================================================================
Dep. Variable:         active_transit   No. Observations:              1952156
Model:                          Logit   Df Residuals:                  1952150
Method:                           MLE   Df Model:                            5
Date:                Sun, 02 Feb 2025   Pseudo R-squ.:                 0.08313
Time:                        12:29:19   Log-Likelihood:            -1.1731e+06
converged:                       True   LL-Null:                   -1.2795e+06
Covariance Type:            nonrobust   LLR p-value:                     0.000
====================================================================================
                       coef    std err          z      P>|z|      [0.025      0.975]
------------------------------------------------------------------------------------
extra_zonal          0.4142      0.006     69.365      0.000       0.402       0.426
dist_raw_km         -0.6854      0.002   -452.541      0.000      -0.688      -0.682
medinc_Z_o           0.1106      0.002     50.287      0.000       0.106       0.115
medinc_Z_d           0.0925      0.002     41.937      0.000       0.088       0.097
BG_active_share      2.5511      0.012    210.040      0.000       2.527       2.575
BG_transit_share     1.9649      0.055     35.938      0.000       1.858       2.072
====================================================================================
"""

In [113]:
#models_no_bg_share['nyc']['summary']
#models_no_bg_share['nyc']['summary']
#models_no_bg_share['phi']['summary']
#models_no_bg_share['sfba']['summary']
models['chi']['summary']
#models['phi']['summary']
#models['nyc']['summary']
#models['sfba']['summary']

<class 'statsmodels.iolib.summary.Summary'>
"""
                           Logit Regression Results                           
==============================================================================
Dep. Variable:         active_transit   No. Observations:              4973005
Model:                          Logit   Df Residuals:                  4972999
Method:                           MLE   Df Model:                            5
Date:                Sun, 02 Feb 2025   Pseudo R-squ.:                 0.09588
Time:                        12:53:32   Log-Likelihood:            -2.5431e+06
converged:                       True   LL-Null:                   -2.8128e+06
Covariance Type:            nonrobust   LLR p-value:                     0.000
====================================================================================
                       coef    std err          z      P>|z|      [0.025      0.975]
------------------------------------------------------------------------------------
extra_zonal          0.2507      0.004     55.875      0.000       0.242       0.259
dist_raw_km         -0.8340      0.001   -856.932      0.000      -0.836      -0.832
medinc_Z_o           0.0764      0.001     54.869      0.000       0.074       0.079
medinc_Z_d           0.0056      0.001      3.989      0.000       0.003       0.008
BG_active_share      1.8752      0.013    143.307      0.000       1.850       1.901
BG_transit_share     7.6041      0.037    203.957      0.000       7.531       7.677
====================================================================================
"""

In [ ]:
# import seaborn as sns

# #sns.scatterplot(x=dfs_add['chi']['dist_km'], y=dfs_add['chi']['dist_raw'])
# dfs_add['chi'][dfs_add['chi']['fips12_o'] != dfs_add['chi']['fips12_d']]

# #sns.scatterplot(x=dfs_add['sfba']['dist_km'], y=dfs_add['sfba']['dist_raw'])
# #dfs_add['sfba'][dfs_add['sfba']['dist_raw'] != 0]
# #bg_dists['sfba'].reset_index()
dfs_raw['chi'][dfs_raw['chi']['fips12_o'] != dfs_raw['chi']['fips12_d']]

,activity_id,fips12_o,fips12_d,primary_mode,zoneID_o,zoneID_d,active_transit,dist_km,extra_zonal,metro_o,...,medinc_Z_o,metro_d,fips5_d,total_pop_d,medinc_d,veh_pp_d,veh_commute_pp_d,medinc_Z_d,BG_active_share,BG_transit_share
0,8.850336e+18,170310101001,170310102021,walking,46,46,1.0,0.965123,0,chi,...,-0.328093,chi,17031,1437,46319,0.312457,0.201227,-1.023910,0.244418,0.057586
1,1.044257e+19,170310101001,170310103001,private_auto,46,46,0.0,1.158021,0,chi,...,-0.328093,chi,17031,766,54130,0.391855,0.250357,-0.869295,0.244418,0.057586
2,7.295663e+18,170310101001,170310105032,private_auto,46,46,0.0,2.402321,0,chi,...,-0.328093,chi,17031,945,66750,0.377778,0.147679,-0.619488,0.244418,0.057586
5,7.156214e+18,170310101002,170310102021,auto_passenger,46,46,0.0,0.853061,0,chi,...,-1.329815,chi,17031,1437,46319,0.312457,0.201227,-1.023910,0.346227,0.073214
6,3.776712e+18,170310101002,170310102021,auto_passenger,46,46,0.0,0.644689,0,chi,...,-1.329815,chi,17031,1437,46319,0.312457,0.201227,-1.023910,0.346227,0.073214
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
7636262,1.552497e+19,170319801001,170318352001,private_auto,80,80,0.0,3.182866,0,chi,...,-0.321640,chi,17031,2228,79000,0.495961,0.350090,-0.377005,0.149023,0.034957
7636263,9.964096e+18,170319801001,170318352001,walking,80,80,1.0,2.017689,0,chi,...,-0.321640,chi,17031,2228,79000,0.495961,0.350090,-0.377005,0.149023,0.034957
7636264,1.108034e+18,170319801001,170318352001,private_auto,80,80,0.0,3.824880,0,chi,...,-0.321640,chi,17031,2228,79000,0.495961,0.350090,-0.377005,0.149023,0.034957
7636265,8.175586e+18,170319801001,170318352001,public_transit,80,80,NaN,2.548705,0,chi,...,-0.321640,chi,17031,2228,79000,0.495961,0.350090,-0.377005,0.149023,0.034957
